# GRAVITY data reduction process: from raw data to science results


**Pablo Marcos-Arenal**

Centro de Astrobiología (INTA-CSIC)


This file intends to cover the tables of the paper related to the near-infrared K- band observations with the GRAVITY interferometer at Very Large Telescope Interferometer (VLTI) at the Paranal Observatory. 

These observations of Herbig Ae/Be stars provide with spatially and spectrally resolved data of their inner disks.



In [1]:
# This line configures matplotlib to show figures embedded in the notebook, 
# instead of opening a new window for each figure. More about that later. 
# If you are using an old version of IPython, try using '%pylab inline' instead.

%matplotlib inline

#%matplotlib qt

#Import modules
import numpy as np
import pandas as pd



# Table A1: Observations


In [3]:
table1_draft = pd.read_csv("./tables/table1_draft.csv", sep=';', header=[0,1])

#Add calibrator info to table
table1_draft.insert(1, "Calibrator", ['HD259163', 'HD78958','HD94533', 'HD131662', 'HD145664'], True) 
# Declare a list that is to be converted into a column 
#calibrator = ['Delhi', 'Bangalore', 'Chennai', 'Patna'] 
  
# Using 'Address' as the column name 
# and equating it to the list 
#df['Calibrator'] = calibrator 

print(table1_draft.to_markdown(showindex=False))

print(table1_draft.to_latex(index=False))



| ('Object', ' ')   | ('Calibrator', '')   | ('RA', '(h:m:s)')   | ('DEC', '(deg:m:s)')   | ('Observation date', ' ')   | ('Type', ' ')   | ('Seeing', ' ')   | ('Telescope', ' ')   |
|:------------------|:---------------------|:--------------------|:-----------------------|:----------------------------|:----------------|:------------------|:---------------------|
| V590Mon           | HD259163             | 06:40:44.6          | +09:48:02              | 27-10-2018                  | SINGLE          | 0.42''            | U1-U2-U3-U4          |
| PDS281            | HD78958              | 08:55:45.9          | -44:25:14              | 20-11-2018                  | SINGLE          | 0.57''            | U1-U2-U3-U4          |
| HD94509           | HD94533              | 10:53:27.2          | -58:25:24              | 20-12-2018                  | SINGLE          | 0.36''            | U1-U2-U3-U4          |
| DGCir             | HD131662             | 15:03:23.8          | -63:22:59         

| ('Object', ' ')   | ('Calibrator', '')   | ('RA', '(h:m:s)')   | ('DEC', '(deg:m:s)')   | ('Observation date', ' ')   | ('Type', ' ')   | ('Seeing', ' ')   | ('Telescope', ' ')   |
|:------------------|:---------------------|:--------------------|:-----------------------|:----------------------------|:----------------|:------------------|:---------------------|
| V590Mon           | HD259163             | 06:40:44.6          | +09:48:02              | 27-10-2018                  | SINGLE          | 0.42''            | U1-U2-U3-U4          |
| PDS281            | HD78958              | 08:55:45.9          | -44:25:14              | 20-11-2018                  | SINGLE          | 0.57''            | U1-U2-U3-U4          |
| HD94509           | HD94533              | 10:53:27.2          | -58:25:24              | 20-12-2018                  | SINGLE          | 0.36''            | U1-U2-U3-U4          |
| DGCir             | HD131662             | 15:03:23.8          | -63:22:59              | 22-01-2019                  | SINGLE          | 0.46''            | U1-U2-U3-U4          |
| HD141926          | HD145664             | 15:54:21.8          | -55:19:44              | 21-03-2019                  | SINGLE          | 0.45''            | A0-B2-C1-D0          |
